In [ ]:
#Importacion de librerias necesarias 
import requests
import json
import string
from virus_total_apis import PublicApi
import pandas as pd

dataframe = pd.DataFrame(columns=['hash'])  #creacion del dataframe

dataframe.head()
def datos(hashX):
    m=hashX.split(",")
    r=int(m[-1])
    hashS =m[0]
    global dataframe
    df = pd.DataFrame() 
    url = 'https://www.virustotal.com/vtapi/v2/file/report'
    hashB = hashS #modificar
    params = {'apikey': 'YOUR KEY', 'resource': hashB, 'allinfo' :'true'}
    df['hash'] = [hashB]
    
    #consulta a la API de virus total y conversion de la respuesta de json a diccionario
    response1 = requests.get(url, params=params)
    convert = json.loads(response1.content)
    
    #librerias importadas inicialmente por el PE
    try:
        numerolibreriasPE=len(convert['additional_info']['imports'].keys())
        df['numerolibreriasPE'] = [numerolibreriasPE]
        if numerolibreriasPE>0:
            for i in convert['additional_info']['imports'].keys(): 
                df[i.lower()] = 1
    except KeyError:
        df['numerolibreriasPE'] = 0
    
    
    #tags importadas 
    try:
        numeroTags=len(convert['tags'])
        df['numeroTags'] = [numeroTags]
        if numeroTags>0:
            for i in convert['tags']: 
                df[i] = 1
    except KeyError:
        df['numeroTags'] = 0
    
    
    #embedded_domains
    try: 
        convert["additional_info"]["embedded_domains"]
        numeroDominiosEmb = len(convert['additional_info']['embedded_domains'])
        #dominios a la red .tor
        if numeroDominiosEmb>0: 
            StrAA = "".join(convert['additional_info']['embedded_domains'])
            if StrAA.find('.onion') == -1: 
                dominioEmbTor=0
            else: 
                dominioEmbTor=1

        #agregar a dataframe
        df['NumeroDominiosEmbebidos'] = [numeroDominiosEmb]
        df['DominiosEmbebidosTor'] = [dominioEmbTor]
    except KeyError: 
        df['NumeroDominiosEmbebidos'] = 0
        df['DominiosEmbebidosTor'] = 0
    
    # numero de lenguajes en el ejecutable (PE)
    try:
        numeroLenguajes = len(convert['additional_info']['pe-resource-langs'])
        df['numeroLenguajes'] = [numeroLenguajes]
        #numero de cadenas de los diferentes lenguajes
        if numeroLenguajes>0:
            for i in convert['additional_info']['pe-resource-langs'].keys():
                i = i.split(" ")
                a = i[0].lower()
                df[a] = 1 
    except KeyError:             
        df['numeroLenguajes'] = 0
    
    #dominios contactados
    try: 
        numeroDominiosContactados=len(convert['additional_info']['contacted_domains'])
        df['numeroDominiosContactados'] = [numeroDominiosContactados]
        #dominios a la red .tor
        if numeroDominiosContactados>0: 
            StrA = "".join(convert['additional_info']['contacted_domains'])
            if StrA.find('.onion') == -1: 
                dominioTor=0
            else: 
                dominioTor=1
        df['numeroDominiosContactados'] = [numeroDominiosContactados]
        df['DominiosContactadosTor'] = [dominioTor]
    except KeyError:
        df['numeroDominiosContactados'] = 0
        df['DominiosContactadosTor'] = 0
    
    
    #URL contactadas
    try: 
        numeroURLContactadas=len(convert['additional_info']['contacted_urls'])
        #URLS a la red .tor
        dominioTor=0
        if numeroURLContactadas>0: 
            StrA = "".join(convert['additional_info']['contacted_urls'])
            if StrA.find('.onion') == -1: 
                dominioTor=0
            else: 
                dominioTor=1
        df['numeroURLContactadas'] = [numeroURLContactadas]       
        df['URLContactadosTor'] = [dominioTor]
    except KeyError:
        df['numeroURLContactadas'] = 0       
        df['URLContactadosTor'] = 0
   

    #IPs contactadas
    try: 
        numeroIPContactadas=len(convert['additional_info']['contacted_ips'])
        df['numeroIPContactadas'] = [numeroIPContactadas]
    except KeyError:
        df['numeroIPContactadas'] = 0       



    #conexiones en tiempo de ejecucion
    try:
        convert['additional_info']['behaviour-v1']['network']
        #numero de conexiones UDP 
        NumConexionesUDP=len(convert['additional_info']['behaviour-v1']['network']['udp'])
        df['NumConexionesUDP'] = [NumConexionesUDP]
        NumConexionesHTTP=len(convert['additional_info']['behaviour-v1']['network']['http'])
        df['NumConexionesHTTP'] = [NumConexionesHTTP]
        NumConexionesTCP=len(convert['additional_info']['behaviour-v1']['network']['tcp'])
        df['NumConexionesTCP'] = [NumConexionesTCP]
        NumConexionesDNS=len(convert['additional_info']['behaviour-v1']['network']['dns'])
        df['NumConexionesDNS'] = [NumConexionesDNS]
    except KeyError:
        df['NumConexionesUDP'] = 0
        df['NumConexionesHTTP'] = 0
        df['NumConexionesTCP'] = 0
        df['NumConexionesDNS'] = 0

    
    #Operaciones con archivos
    try:
        NumArchivosAbiertos=len(convert['additional_info']['behaviour-v1']['filesystem']['opened'])
        NumArchivosLeidos=len(convert['additional_info']['behaviour-v1']['filesystem']['read'])
        NumArchivosMovidos=len(convert['additional_info']['behaviour-v1']['filesystem']['moved'])
        NumArchivosDescargados=len(convert['additional_info']['behaviour-v1']['filesystem']['downloaded'])
        NumArchivosEscritos=len(convert['additional_info']['behaviour-v1']['filesystem']['written'])
        NumArchivosReemplazados=len(convert['additional_info']['behaviour-v1']['filesystem']['replaced'])
        NumArchivosBorrados=len(convert['additional_info']['behaviour-v1']['filesystem']['deleted'])
        NumArchivosCopiados=len(convert['additional_info']['behaviour-v1']['filesystem']['copied'])
        df['NumArchivosAbiertos'] = [NumArchivosAbiertos]
        df['NumArchivosLeidos'] = [NumArchivosLeidos]
        df['NumArchivosMovidos'] = [NumArchivosMovidos]
        df['NumArchivosDescargados'] = [NumArchivosDescargados]
        df['NumArchivosEscritos'] = [NumArchivosEscritos]
        df['NumArchivosReemplazados'] = [NumArchivosReemplazados]
        df['NumArchivosBorrados'] = [NumArchivosBorrados]
        df['NumArchivosCopiados'] = [NumArchivosCopiados]
    except KeyError:
        df['NumArchivosAbiertos'] = 0
        df['NumArchivosLeidos'] = 0
        df['NumArchivosMovidos'] = 0
        df['NumArchivosDescargados'] = 0
        df['NumArchivosEscritos'] = 0
        df['NumArchivosReemplazados'] = 0
        df['NumArchivosBorrados'] = 0
        df['NumArchivosCopiados'] = 0

        
 
    #Librerias importadas en tiempo de ejecucion 
    try:
        numeroLibreriasEJ = len(convert['additional_info']['behaviour-v1']['runtime-dlls'])
        df['numeroLibreriasEJ'] = [numeroLibreriasEJ]
        if numeroLibreriasEJ>0:
            for i in range (0, len(convert['additional_info']['behaviour-v1']['runtime-dlls'])):
                a =convert['additional_info']['behaviour-v1']['runtime-dlls'][i]
                b = dict(a)
                cadena = "".join(b.get('file'))
                if cadena.find('\\') != -1:
                    cadena = cadena.split("\\")
                    df[cadena[-1].lower()] = 1
                else:
                    df[b.get('file')] = 1
    except KeyError:
        df['numeroLibreriasEJ'] = 0


        
        
    #Operaciones con procesos en tiempo de ejecucion   
    try:
        numeroProcesosCreados = len(convert['additional_info']['behaviour-v1']['process']['created'])
        numeroProcesosInjectados = len(convert['additional_info']['behaviour-v1']['process']['injected'])
        numeroProcesosTerminados = len(convert['additional_info']['behaviour-v1']['process']['terminated'])
        numeroProcesosShell = len(convert['additional_info']['behaviour-v1']['process']['shellcmds'])
        df['numeroProcesosCreados'] = [numeroProcesosCreados]
        df['numeroProcesosInjectados'] = [numeroProcesosInjectados]
        df['numeroProcesosTerminados'] = [numeroProcesosTerminados]
        df['numeroProcesosShell'] = [numeroProcesosShell]
    except KeyError:
        df['numeroProcesosCreados'] = 0
        df['numeroProcesosInjectados'] = 0
        df['numeroProcesosTerminados'] = 0
        df['numeroProcesosShell'] = 0   

    #utilizacion de objetos mutex

    try:
        mutexAbiertos = len(convert['additional_info']['behaviour-v1']['mutex']['opened'])
        df['mutexAbiertos'] = [mutexAbiertos]
        if mutexAbiertos>0: 
            for i in range (0, len(convert['additional_info']['behaviour-v1']['mutex']['opened'])):
                a = convert['additional_info']['behaviour-v1']['mutex']['opened'][i]
                b = dict(a)
                df[b.get('mutex')] = 1
        mutexCreados = len(convert['additional_info']['behaviour-v1']['mutex']['created'])
        df['mutexCreados'] = [mutexCreados]
        if mutexCreados >0: 
            for i in range (0, len(convert['additional_info']['behaviour-v1']['mutex']['created'])):
                a = convert['additional_info']['behaviour-v1']['mutex']['created'][i]
                b = dict(a)
                df[b.get('mutex')] = 1
    except KeyError:
        df['mutexAbiertos'] = 0
        df['mutexCreados'] = 0

    #creacion de persistencia
    try:
        persistencia = len(convert['additional_info']['autostart'])
        if persistencia>0:
            df['persistencia'] = 1
        else:
            df['persistencia'] = 0
    except KeyError:
        df['persistencia'] = 0        

    
    #Manejo de hocks

    try:
        hooking = len(convert['additional_info']['behaviour-v1']['hooking'])
        df['hooking'] = [hooking]
        if hooking>0: 
            for i in range (0, len(convert['additional_info']['behaviour-v1']['hooking'])):
                a = convert['additional_info']['behaviour-v1']['hooking'][i]
                b = dict(a)
                df[b.get('type')] = 1
    except KeyError:
        df['hooking'] = 0        

   

    #tags de las capacidades en tiempo de ejecucion

    try:
        capacidades= len(convert['additional_info']['capabilities_tags'])
        df['capacidades'] = [capacidades]
        if capacidades>0: 
            for i in convert['additional_info']['capabilities_tags']: 
                df[i] = 1
    except KeyError:
        df['capacidades'] = 0    
    except AttributeError:
        df['capacidades'] = 0     

        
    #modificaciones del registro en tiempo real
    try:
        registroBorrado= len(convert['additional_info']['behaviour-v1']['registry']['deleted'])
        df['registroBorrado'] = [registroBorrado]
        registroInsertado= len(convert['additional_info']['behaviour-v1']['registry']['set'])
        df['registroInsertado'] = [registroInsertado]
    except KeyError:
        df['registroBorrado'] = 0
        df['registroInsertado'] = 0
        
    #Es o no ransomware
    try:
        if r == 1:
            df['ransomware'] = 1
        else: 
            df['ransomware'] = 0
    except KeyError:
            df['ransomware'] = 2

    
    #Type que clase de ejecutable es
    try:
        df['Tipo'] = convert['type']
    except KeyError:
        df['Tipo'] = ['desconocido']

    #integracion de la informacion extraida al dataframe general 
    dataframe = dataframe.append(df,ignore_index=True)    
    


In [ ]:
dataframe.head()

In [ ]:
#Leer los hashes para armar el archivo dataframe

f = open ('D:/escritorio/has1.txt.txt','r')
j = 0
m = 0
for i in f:
    datos(i)
    j = j + 1
    m = m + 1
    if j == 1000:
        print("procesado actualmente: ", m)
        j = 0
f.close()
print("Proceso de descarga de informacion terminado")

In [ ]:
#Cambiar valores NaN por cero (0)

dataframe.fillna(0, inplace=True)

In [ ]:
#Pruebas Se puede borrar

dataframe.head(20)


Despues de la ejecion del script tenemos un total de 7596 columnas o caracteristicas de 19507 ejecutables entre ransomware y programas extraidos de windows 10. 

In [ ]:
#Guardar los datos en un archivo .csv para poder trabajar con ellos con mas calma sin la necesidad de volver a ejecutar el script 
#completamente. 

dataframe.to_csv('D:\escritorio\data.csv', header=True)

**Limpieza de datos**

En este apartado se empezará a realizar la limpieza de los datos de acuerdo a la descarga de la información que se realizó en de virustotal.

In [1]:
#lectura del archivo en donde se guardo la informacion. 

import pandas
filename = 'D:\escritorio\Muestra\Datos\data - copia.csv'
data = pandas.read_csv(filename, header=0)

print(data.shape)19



(19507, 7587)


### Analisis de la información descargada

Conteo de la categoria tipo, en la cual se pueden ver que existen **13748** valores Win32 EXE, los cuales corresponden a los ejecutables de windows 32. que son los que nos sirven para comprar los archivos extraidos del Windows. 

In [2]:
data['Tipo'].value_counts() 


Win32 EXE                   13748
Win32 DLL                    3146
unknown                      1138
desconocido                   432
Text                          209
ZIP                           157
HTML                          156
Google Chrome Extension       153
C++                           115
JavaScript                     96
DOS EXE                        35
RAR                            34
Windows shortcut               31
CAB                            13
XML                             8
DOS COM                         6
GZIP                            6
Dyalog                          3
Android                         3
C                               3
Email                           2
ICO                             2
Flash                           2
PDF                             2
7ZIP                            2
Office Open XML Document        2
COFF                            1
Network capture                 1
JAR                             1
Name: Tipo, dt

En total tenemos 13748 muestras de ransomware que corresponden a archivos **win32** los cuales son los que utilizaremos para el entrenamiento del modelo. No se tendra en cuenta los archivos, de otras categorias ya que la gran mayoria de archivos de los archivos extraidos de windows 10 son: Win32 EXE

Detalle de archivos de windows 10: 

| Tipo | No. | 
| :- | -: | 
| Win32 EXE  | 2002 | 
| unknown  | 988 |
| desconocido  | 432 |
| Win32 DLL   | 41 |
| DOS EXE  | 8 |
| DOS COM  | 5 |
| Dyalog  | 3 |
| ICO  | 2 |
| COFF  | 1 |

No todos los archivos analizados por la sandbox son clasificados como win32 realizando el filtrado y analisis se puede ver a que corresponden las demas categorias: 

### Analisis de los archivos desconocidos y unknown segun la informacion extraida. 

In [ ]:
#analisis de los archivos desconocidos y unknown segun la informacion extraida. 

data[(data['Tipo']=='unknown') & (data['ransomware']==0)]['hash'].head(10)


La siguiente imagen extraida de virustotal demuestra que efectivamente los archivos clasificados como **unknown** no tienen analisis, ya sea porque no se pudo efectuar, o porque por sus caracteristicas no se realizó. 

<img src="https://1.bp.blogspot.com/-4VRjSSO5HEI/Xw0PMnhZH1I/AAAAAAAABx0/HnqpRqn5bdUZtFe6haVY7jqB3pf6yt3vQCLcBGAsYHQ/s1600/analisis1.png">

Igualmente los archivos clasificados como **desconocido** no han sido analizados esto ocurre porque hasta la fecha no se tiene registro de su subida a la plataforma para realizar el posterior analisis; posibles razones ejecutables de los cuales nadie ha encontrado un comportamiento sospechoso. Adicionalmente los archivos "Win32 DLL" traen informacion basica de su analisis pero ninguno es analizado en una sandbox por lo cual traen informacion incompleta. 

Esto mismo pasa con los archivos, DOS EXE, DOS COM, Dyalog, ICO, COFF. 


**Conclusión:** Unicamente nos sirven los tipo **Win32 EXE** para poder realizar una buena comparación. 

In [ ]:
#creación de un nuevo dataframe unicamente con los datos win32 EXE
datos_filtered = data[data['Tipo'] == 'Win32 EXE']
datos_filtered.shape

In [ ]:
# de acuerdo a la eliminacion de los valores que no generan mucho valor, ahora tenemos la siguiete cantidad de datos: 
datos_filtered['ransomware'].value_counts()

Cantidad de muestras de malware por tipo despues de la depuración inicial: 

| Tipo| Numero | 
| :- | -: | 
| Ransomware  | 11746 | 
| Benigno  | 2002 |

Teniendo en cuenta que aun tenemos 7587 caracteristicas. 

### Depuración de columnas

Ya que los datos fueron descargados de una fuente externa muchas de las características pueden contener datos "basura" o que no aporten mucho valor al análisis posterior por lo cual el trabajo inicial será buscar esas columnas o características que no generan valor para eliminarlas. 

In [ ]:
datos_filtered.describe()

In [ ]:
dfColumnas = datos_filtered.columns
print(dfColumnas)
dfColumnas.shape

In [ ]:
datos_filtered.to_csv('D:\escritorio\data2.csv', header=True)

In [1]:
import pandas
filename = 'D:\escritorio\data2.csv'
dataLimpieza = pandas.read_csv(filename, header=0)

print(dataLimpieza.shape)

(13748, 7588)


In [2]:
dfColumnas = dataLimpieza.columns
print(dfColumnas)
dfColumnas.shape

Index(['Unnamed: 0', 'Unnamed: 0.1', 'Unnamed: 1', '!#@tr$*#', '!@#@drhg',
       '#qhbu7h6ab', '$AZ', '$OK', '$ZR', '((Mutex))',
       ...
       '{D9961D0B-0106-5584-AD6D-884HSI64CNI9}',
       '{DBC7574C-4498-803C-3B52-A6F482B3FDE5}',
       '{ECCC50B5-064A-4693-B104-925714A4C74B}',
       '{EEEB680D-AE62-4375-B93E-E9AE5FF585C1}',
       '{a20cd692-8e41-11e1-9999-806d6172696f}',
       '{a4c6366c-be41-67d2-a73d-db175e228648}',
       '{ca904f65-e663-6db9-8e9c-9deb8187a20e}',
       '{cffa1ea4-e207-6e2d-9d41-a95950155215}',
       '{e8600a57-1dc4-63f6-8d1b-555564a5a02a}', '~'],
      dtype='object', length=7588)


(7588,)

### Depuración de Columnas

Vemos las columnas que no tienen datos, esto se debe a la elimnacion anterior de las muestras clasificadas como no **"Win32 EXE"** en total 58 columnas. 

In [3]:
#el siguiente código muestra la cantidad de columnas sin datos y realiza la eliminación de las mismas. 
j = 0
for i in dfColumnas:
    if dataLimpieza[i].sum() == 0:
        j+=1
        print(i)
        dataLimpieza.drop([i],axis='columns', inplace=True)
print("\n\nNumero de columnas eliminadas: ",j)


7zip
NumArchivosDescargados
acledit.dll
acroform
android
apk
autoaction
c
cab
cap
coff
com
contains-elf
contains-pe
cpp
create-ole
crx
cve-2008-2992
cve-2013-0074
cve-2018-5002
docx
dyalog
dyn-calls
email
encrypted
ess
ext-interface
flash
gzip
handle-file
html
ico
invalid-xref
jar
javascript
js-embedded
lnk
macros
msstpapi.dll
mz
obfuscated
open-file
pdf
pedll
rar
rbuffa
reflection
rtm.dll
software-collection
tar-bundle
telephony
text
winfax.dll
xml
xpcom.dll
zero-filled
zip
zlib


Numero de columnas eliminadas:  58


In [4]:
dataLimpieza.shape

(13748, 7530)

In [6]:
#Eliminacion de la columna "Unnamed: 0" que unicamente corresponde a un indice. 
dataLimpieza['Unnamed: 0'].head(10)

0     0
1     1
2     2
3     3
4     4
5     5
6     6
7     8
8     9
9    12
Name: Unnamed: 0, dtype: int64

In [7]:
dataLimpieza.drop(['Unnamed: 0'],axis='columns', inplace=True)

In [8]:
#realicemos la comprobacion de los datos que pueden no tener datos tipo float  
dfColumnas = dataLimpieza.columns
a = 0
for i in dfColumnas: 
    if dataLimpieza[i].dtypes == 'float64':
        a+=1
    else:
        print(i)
   

Unnamed: 0.1
Tipo
hash


Efectivamente comprobamos que el dataframe unicamente tiene dos columnas con tipos de datos diferentes a float64 la columna hash que debe ser eliminada antes del entrenamiento y la columna Tipo la cual es la columna objetivo y debe ser convertida. 

In [9]:
dataLimpieza['Unnamed: 0.1'].head(10)
dataLimpieza.drop(['Unnamed: 0.1'],axis='columns', inplace=True)

In [10]:
dataLimpieza.shape

(13748, 7528)

In [11]:
#Eliminación de la columna hash, datos que no usaremos en el entrenamiento. 
dataLimpieza.drop(['hash'],axis='columns', inplace=True)

In [16]:
#guardar los datos ya limpios para el trabajo con la red neuronal. 
dataLimpieza.to_csv('D:\escritorio\data3.csv', header=True)